```
Copyright 2025 IBM Corporation

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Credit Card Fraud Detection using RNN Model on Triton Inference Server with `PyTorch` Backend

## Overview

This notebook demonstrates how to deploy a pre-trained credit card fraud detection model using the PyTorch backend of [IBM Z Accelerated for NVIDIA Triton™ Inference Server](https://github.com/IBM/ibmz-accelerated-for-nvidia-triton-inference-server). It focuses on deployment and inference using a custom `model.py` file to load the `.pt` model and define architecture logic. 

The dataset used for training includes transaction-level features such as `User`, `Card`, `Time`, `Amount`, `merchant details`, and a binary `fraud label`. A recurrent neural network (RNN) model was trained using sequences of these features. While **data preprocessing, training, and saving** the model are not part of this notebook, a high-level reference is provided for context. The core objective here is to demonstrate the deployment process of the trained model for inference.


### Pre-requisites
Please ensure the following tools and packages are available in your environment:

**Required Tools:**
 - Docker
 - IBM Cloud account
 - Python 3.10+

**Python Dependencies:**
 - jupyter
 - pip
 - torch
 - numpy
 - requests

**Model Requirement:**
 - Trained model in PyTorch `.pt` format on Credit Card Fraud Detection dataset.

Refer to the [credit-card-fraud notebook](https://github.com/IBM/ibmz-accelerated-for-pytorch/tree/main/samples/credit-card-fraud) to train and save the model. This will generate a .pt file needed for deployment.

### Model Summary
The model is trained to detect fraudulent credit card transactions using a preprocessed dataset containing the following columns:
`User, Card, Year, Month, Day, Time, Amount, Use Chip, Merchant Name, Merchant City, Merchant State, Zip, MCC, Errors?, Is Fraud?`

Preprocessing was done using `TransformerPipeline` with the following components:

1. Categorical Columns: Imputation and encoding (Merchant State, etc.)
2. Temporal Columns: Normalization (Year, Month, Day, Time)
3. Amount: Transformation and scaling

The trained model consists of:

1. LSTM layers with sequence length = 7 and hidden size = 200
2. Saved in PyTorch (.pt) format and then exported to ONNX for deployment
For full details on data preprocessing, model training, and export to ONNX format, refer to this repository [here](https://github.com/IBM/ibmz-accelerated-for-pytorch/tree/main/samples/credit-card-fraud)

#### 1. Define Model Repository Structure:
Let’s assume the model is named `fraud_detection_pytorch`, and the model repository directory structure as below: 
```
model_repository/
└── fraud_detection_pytorch/
    ├── 1/
    │   └── model.pt
    |   └── model.py
    └── config.pbtxt
```

In [1]:
import os
model_name = "fraud_detection_pytorch"
model_repo = "model_repository"
model_version = "1"

# Create directories
# Create the model repository directory. The name of this directory is arbitrary.
REPO_PATH = os.path.abspath('model_repository')

base_path = os.path.join(model_repo, model_name)
version_path = os.path.join(base_path, model_version)

os.makedirs(version_path, exist_ok=True)

**NOTE:** Update the model path in the notebook as required.

In [2]:
#Copying the trained model saved in .pt to the model repository path
#Set the PyTorch model path
model_path = "../CCF_LSTM_Trained_Model_Inputs/lstm.pt"

import shutil

# Copy the file (keeps the same filename)
# shutil.copy(model_path, os.path.join(version_path,"model.pt"))
shutil.copy(model_path, version_path)

'model_repository/fraud_detection_pytorch/1/lstm.pt'

#### 2. Define Model Architecture in `model.py`:

Create a model.py that:
 - Defines the RNN architecture (same as training time),
 - Loads the `.pt` model in the constructor,
 - Implements the forward() method to handle prediction logic.

**Note:** Since this uses the Python backend, you only need to implement the `__init__` and `forward()` methods. `initialize()` and `finalize()` are not required.

In [3]:
model_py_content = """import torch

class FraudDetectionModel(torch.nn.Module):
    def __init__(self):
        self.input_size = 220  # Features per time step
        self.hidden_size = 200  # LSTM hidden layer size
        
        super(FraudDetectionModel, self).__init__()

        self.rnn = torch.nn.LSTM(self.input_size, self.hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(self.hidden_size, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        return self.sig(self.fc(out))
"""

with open(os.path.join(version_path, "model.py"), "w") as f:
    f.write(model_py_content)

#### 3. The Model Configuration File Preparation:
The configuration file for a model provides Triton with key details about the model and specifies how it should be used. While you can explore all general Triton configuration options [here](https://github.com/triton-inference-server/server/blob/master/docs/model_configuration.md). The configuration options are specific to the **pytorch** backend. This example focuses on a few of the most common and essential options.

Below are general descriptions of these options:
- **backend**: This parameter must be provided as “pytorch” while utilising the PyTorch Backend.
- **max_batch_size**: The maximum batch size can be passed to this model. In general, the only limit on the size of batches passed to an pytorch backend is the memory available with which to process them.
- **input**: Options in this section tell Triton the number of features to expect for each input sample.
- **output**: Options in this section tell Triton how many output values there will be for each sample.
- **instance_group**: This determines how many instances of this model will be created.
- **dynamic_batching**: This allows the server to combine multiple inference requests into a single batch, optimizing NNPA or CPU utilization and improving throughput.

Based on this information, let's set up configuration files for our models.

In [4]:
config_pbtxt = f"""name: "{model_name}"
backend: "pytorch"
max_batch_size: 32
input {{
  name: "input_features"
  data_type: TYPE_FP32
  dims: [7, 220]
}}

output {{
  name: "is_fraud"
  data_type: TYPE_FP32
  dims: [1]
}}

instance_group {{
  count: 2
  kind: KIND_CPU
}}

dynamic_batching {{
  preferred_batch_size: 32
  max_queue_delay_microseconds: 250000
}}
"""

with open(os.path.join(base_path, "config.pbtxt"), "w") as f:
    f.write(config_pbtxt)

#### Summary: Folder Structure Created

In [5]:
from pathlib import Path

def print_tree(path):
    print(f"{path}/")
    for p in sorted(Path(path).rglob('*')):
        depth = len(p.relative_to(path).parts)
        spacer = '    ' * depth
        print(f"{spacer}{p.name}")

print_tree(REPO_PATH)

/home/hemaprasannakc/kanaka/demo/pytorch-backend/model_repository/
    fraud_detection_pytorch
        1
            lstm.pt
            model.py
        config.pbtxt


In [6]:
# View Model configuration
from IPython.display import Code

with open(os.path.join(base_path, "config.pbtxt"), 'r') as f:
    display(Code(f.read(), language='protobuf'))

name: "fraud_detection_pytorch"
backend: "pytorch"
max_batch_size: 32
input {
  name: "input_features"
  data_type: TYPE_FP32
  dims: [7, 220]
}

output {
  name: "is_fraud"
  data_type: TYPE_FP32
  dims: [1]
}

instance_group {
  count: 2
  kind: KIND_CPU
}

dynamic_batching {
  preferred_batch_size: 32
  max_queue_delay_microseconds: 250000
}

#### 4. Deploy and Infer the Models on IBM Z Accelerated for NVIDIA Triton™ Inference Server

Deploy the model for real-time inference using Triton's `pytorch` backend.

#### 4.1 Download the IBM Z Accelerated for NVIDIA Triton™ Inference Server Container image

 - Log in to the IBM Container Registry ([icr.io](https://ibm.github.io/ibm-z-oss-hub/containers/ibmz-accelerated-for-nvidia-triton-inference-server.html)) using valid credentials.
 - Pull the [IBM Z Accelerated for NVIDIA Triton™ Inference Server Container image](https://ibm.github.io/ibm-z-oss-hub/containers/ibmz-accelerated-for-nvidia-triton-inference-server.html).
 - Refer to the IBM documentation for details on obtaining credentials [here](https://ibm.github.io/ibm-z-oss-hub/main/main.html).

In [ ]:
# Replace with your keyname and API key
keyname = "iamapikey"
apikey = "<your-apikey>"
registry_url = "http://icr.io/"

# Perform Docker login using the API key
!echo {apikey} | docker login -u {keyname} {registry_url} --password-stdin

# Pull a Docker image from the registry
TRITON_IMAGE = "icr.io/ibmz/ibmz-accelerated-for-nvidia-triton-inference-server:1.4.0" #Latest version of IBM Z Accelerated for NVIDIA Triton™ Inference Server container image
!docker pull {TRITON_IMAGE}

Login Succeeded


1.4.0: Pulling from ibmz/ibmz-accelerated-for-nvidia-triton-inference-server


2c7366af510b: Pulling fs layer 


0b80c99c521f: Pulling fs layer 


82f34b8a024a: Pulling fs layer 


a2ae4227ce50: Pulling fs layer 


1f4f117092d1: Pulling fs layer 


3b0ff6cadca3: Pulling fs layer 


6ce88ceb0a45: Pulling fs layer 


f46d6ba18537: Pulling fs layer 


3486716f0da8: Pulling fs layer 


5d96a0541b22: Pulling fs layer 


5788b4996ab2: Pulling fs layer 

1f4f117092d1: Waiting 


1f3479c096cd: Pulling fs layer 

3b0ff6cadca3: Waiting 


3f786e57c609: Pulling fs layer 

5d96a0541b22: Waiting 

f46d6ba18537: Waiting 


2c19fac2c1a4: Pulling fs layer 

6ce88ceb0a45: Waiting 

5788b4996ab2: Waiting 


923204231833: Pulling fs layer 

3486716f0da8: Waiting 

3f786e57c609: Waiting 

1f3479c096cd: Waiting 


b0ab249dfe39: Pulling fs layer 

2c19fac2c1a4: Waiting 

a2ae4227ce50: Waiting 

923204231833: Waiting 


6edc9f84dc24: Pulling fs layer 

b0ab249dfe39: Waiting 


5e0ea807c814: Pulling fs layer 


0b80c99c521f: Downloading  149.1kB/14.27MB



82f34b8a024a: Downloading  204.8kB/19.83MB

2c7366af510b: Downloading  292.6kB/28.65MB



0b80c99c521f: Downloading  2.855MB/14.27MB


...


16c449db3d20: Extracting  5.225MB/5.225MB

16c449db3d20: Pull complete 

b2935665fa91: Extracting     239B/239B

b2935665fa91: Extracting     239B/239B



b2935665fa91: Pull complete 

53e56fcc278f: Extracting     290B/290B

53e56fcc278f: Extracting     290B/290B

53e56fcc278f: Pull complete 

3cd6e26ec78c: Extracting  32.77kB/349kB

3cd6e26ec78c: Extracting    349kB/349kB

3cd6e26ec78c: Extracting    349kB/349kB

3cd6e26ec78c: Pull complete 

7915f3cfcd7e: Extracting     425B/425B

7915f3cfcd7e: Extracting     425B/425B

7915f3cfcd7e: Pull complete 

abf503969137: Extracting  229.4kB/19.9MB



abf503969137: Extracting  8.946MB/19.9MB



abf503969137: Extracting  17.66MB/19.9MB



abf503969137: Extracting   19.9MB/19.9MB

abf503969137: Pull complete 
Digest: sha256:484c5f9aeff0c911d110ae85596aa33a5ac3b594f561b712d9cf7440b8a6d8b3
Status: Downloaded newer image for icr.io/ibmz/ibmz-accelerated-for-nvidia-triton-inference-server:1.4.0
icr.io/ibmz/ibmz-accelerated-for-nvidia-triton-inference-server:1.4.0


#### 4.2 Starting the Triton Inference Server
 - Launch the Triton server with the model and configuration files in place.
 - Use Python to verify server readiness and check logs for warnings or errors.

In [8]:
!docker run --shm-size 1G -itd -p 8000:8000 -p 8001:8001 -p 8002:8002 -v {REPO_PATH}:/models --name tritonserver {TRITON_IMAGE} tritonserver --model-repository=/models

8c4539611db978dd04df19d875b9d4826536cd8e61b61bb5b1e6a1e8d7ba58fe


In [9]:
import time
import requests

def check_server_status(server_url, num_retries=30, sleep_time=1):
    """
    Use Triton's ready endpoint to verify that the server and the models are ready for inference.
    Using curl to access the HTTP endpoint that indicates server status.
    Parameters:
        num_retries (int): number of times to send a ready status
                                request to the server before raising
                                an exception
        sleep_time (int): amount of time in seconds to sleep between retries
    Returns:
    bool: False if the server is not ready even after num_retries retries, otherwise True.   
    """

    retries = num_retries
    server_status_api = f"http://{server_url}/v2/health/ready"

    while retries > 0:
        try:
            response = requests.get(server_status_api)
            time.sleep(sleep_time)
            if response.status_code == 200:
                return True
            else:
                retries -= 1
        except requests.RequestException as e:
            time.sleep(sleep_time)
            retries -= 1
            print(f"Server not yet ready, encountered error: {e}")
    return False

In [10]:
#Server Health ready check
HOST = 'localhost'
PORT = 8000

server_status = check_server_status(server_url=f'{HOST}:{PORT}')
if not server_status:
    raise RuntimeError(("Failed to start Triton server. Check the log for details."))
else:
    print("Triton Inference Server started successfully.")

Server not yet ready, encountered error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Server not yet ready, encountered error: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Server not yet ready, encountered error: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Server not yet ready, encountered error: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


Triton Inference Server started successfully.


In [11]:
!docker logs tritonserver

=-=-=-=-=-= IBM Z Accelerated for NVIDIA Triton Inference Server =-=-=-=-=-=-=-=-=-=-=-=-=-=

Version: 1.4.0

License: https://github.com/IBM/ibmz-accelerated-for-nvidia-triton-inference-server#Licenses

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-==-=-=-=-=-=-=-=-=-=-=-=-=-=-=-



I0630 11:36:48.645517 1 model_lifecycle.cc:472] "loading: fraud_detection_pytorch:1"

I0630 11:36:51.215722 1 python_be.cc:1912] "TRITONBACKEND_ModelInstanceInitialize: fraud_detection_pytorch_0_0 (CPU device 0)"

I0630 11:36:51.215748 1 python_be.cc:1912] "TRITONBACKEND_ModelInstanceInitialize: fraud_detection_pytorch_0_1 (CPU device 0)"

I0630 11:36:52.472934 1 model.py:81] "Initializing model instance for 'fraud_detection_pytorch'"

I0630 11:36:52.474158 1 model.py:81] "Initializing model instance for 'fraud_detection_pytorch'"

I0630 11:36:52.474575 1 pytorch_model.py:99] "fraud_detection_pytorch : Using nnpa hardware optimization for inferencing (is_user_specified: False)"

I0630 11:36:

#### 4.3 Submitting inference requests
 - Send inference requests to the Triton server.
 - Validate results by comparing them with local predictions.

In [12]:
import json

def get_response(server_url, input_data_json, model_name, model_version=None):
    
    headers = {
        "Content-Type": "application/json"
    }

    model_version = model_version if model_version is not None else 1
    model_inference_url = f"http://{server_url}/v2/models/{model_name}/versions/{model_version}/infer"

    response = {
        "response_obj" : None,
        "status_code" : None, 
        "error_reason": None
    }
    resp = None

    #load input data
    file = open(input_data_json, 'r')
    payload = json.loads(''.join(file.readlines()).replace(" ", "").replace("\n",""))

    headers = {
        "Content-Type": "application/json"
    }

    print(f"\nMaking inference call to {model_inference_url} with data \n {payload} \n\n ")

    try: 
        start_time = time.time()
        resp = requests.post(
                        model_inference_url,
                        headers=headers,
                        json=payload
                    )
        end_time = time.time()
        elapsed_time = end_time - start_time
        if resp.status_code != 200:
            raise Exception(f"HTTP request failed with {resp.status_code}")
        
        print(f"Inference completed successfully in {elapsed_time:.6f} seconds")
        response["response_obj"] = resp.json()
        response["status_code"] = resp.status_code
        response["elapsed_time"] = elapsed_time
        response["error_reason"] = None

    except Exception as e:
        # still fecth the error info and create a responce object 
        if resp is not None:
            response["response_obj"] = resp.json() 
            response["status_code"] = resp.status_code
        else:
            response["status_code"] = 400 # Client side Exception
        response["error_reason"] = e

        return response
    return response

#### Inference Testing and Measuring Time Consumption of LSTM Model:

In [13]:
#Set the input data in JSON file path
input_data_file = "../CCF_LSTM_Trained_Model_Inputs/input_data.json"
response = get_response(f'{HOST}:{PORT}', input_data_file, model_name)
try: 
    model_op = response["response_obj"]["outputs"][0]["data"]
    cpu_elapsed_time = response["elapsed_time"]
    print(f"The totel time taken by {model_name} is {cpu_elapsed_time}")
    print("Model inference passed for model {model_name}" f" with response {str(model_op)}")
except Exception as e:
    print(f"Model inference failed for model {model_name}"  f"with exception : {e}, \n the server response is {response}")


Making inference call to http://localhost:8000/v2/models/fraud_detection_pytorch/versions/1/infer with data 
 {'inputs': [{'name': 'input_features', 'shape': [2, 7, 220], 'datatype': 'FP32', 'data': [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

Inference completed successfully in 0.254711 seconds
The totel time taken by fraud_detection_pytorch is 0.2547109127044678
Model inference passed for model {model_name} with response [0.4990234375, 0.4990234375]


In [14]:
# Shut down the server
!docker rm -f tritonserver

tritonserver


## Conclusion 

In this notebook, we deployed a trained RNN-based PyTorch model for credit card fraud detection using the PyTorch backend of **IBM Z Accelerated for NVIDIA Triton™ Inference Server**. By leveraging a custom `model.py` handler, we were able to load and serve a .pt model directly without compilation. This makes the PyTorch backend highly flexible and suitable for iterative development and real-time inference scenarios.

The approach showcased here is ideal for:
 - Models requiring custom pre/post-processing logic
 - Use cases with dynamic model behavior
 - Rapid prototyping with minimal setup

This workflow enables faster deployment cycles while retaining the ability to extend logic and optimize inference pipelines.